# Reading a Simulation's History

Author: Sam Eure

April 7, 2025

### Simulation History Feature
The Simulation History feature is designed to automatically log events each time a simulation (`api.endpoints.Simulation`) is created and during key operations on the simulation such as adding a ZTP script or the enabling/disabling of automatic OOB.

### Accessing Simulation History
To retrieve the history associated with a specific simulation, use the `get_history` method available on the `Simulation` object; this returns an `Iterator[api.endpoints.History]`. The following are optional filtering parameters that may be passed to the `get_history` method:
- `category`
- `actor`
- `search`
- `ordering`

### Immutability of History Instances
History instances are immutable and cannot be instantiated directly by clients. This ensures the integrity and consistency of the historical data. As a result, `History` instances may not be refreshed or deleted by clients. 

### Creating and listing `History`

In [43]:
import time

from air_sdk import AirApi
from air_sdk.endpoints import History, Simulation

api = AirApi.with_ngc_config()
# Or use SAK directly
# api = AirApi.with_api_key(api_key='nvapi-xyz')

# Create a simulation with a JSON import
json_data = {
    'format': 'JSON',
    'name': 'JSON Sim',
    'content': {
        'nodes': {
            'node1': {
                'cpu': 2,
                'memory': 1024,
                'storage': 10,
                'os': 'generic/ubuntu2204',
                'cpu_arch': 'x86',
            },
        },
        'oob': False,
    },
}

sim: Simulation = api.simulations.import_from_data(**json_data)
print('waiting for simulation to finish importing...', end='')
while sim.state == 'IMPORTING':
    sim.refresh()
    time.sleep(1)
    print('.', end='')


# Add a ZTP Script
ztp_script_path = '../files/ztp/basic_script.sh'
with open(ztp_script_path, 'r') as f:
    sim.create_ztp_script(content=f.read())

# Enable automatic OOB
sim.enable_auto_oob()

waiting for simulation to finish importing....

Now that a simulation has been created and operations have been performed on the simulation, the history of the simulation may be retreived and viewed.

In [44]:
from typing import Iterator

history_iter: Iterator[History] = sim.get_history(ordering='created')

for history in history_iter:
    print(
        history.actor.ljust(12),
        f'({history.created.strftime("%Y-%m-%d %H:%M:%S.%f")}):',
        history.description,
    )

Jenson Huang (2025-04-07 18:38:09.223522): Simulation `JSON Sim` with ID `2311ab4d-f522-48e9-85c2-0f985e7207d9` created by importing a topology in `JSON` format. Topology is scheduled for validation.
NVIDIA Air   (2025-04-07 18:38:09.227707): Initiating topology validation.
NVIDIA Air   (2025-04-07 18:38:09.231380): Validation complete. Initiating creation of simulation objects.
NVIDIA Air   (2025-04-07 18:38:09.241702): Topology successfully imported.
NVIDIA Air   (2025-04-07 18:38:09.242909): Transitioning from `IMPORTING` to `INACTIVE` state.
Jenson Huang (2025-04-07 18:38:12.266115): ZTP script added.
Jenson Huang (2025-04-07 18:38:14.380638): Enabled automatic OOB


In [50]:
history.dict()

{'object_id': '2311ab4d-f522-48e9-85c2-0f985e7207d9',
 'model': 'simulation',
 'created': datetime.datetime(2025, 4, 7, 18, 38, 14, 380638, tzinfo=datetime.timezone.utc),
 'actor': 'Jenson Huang',
 'description': 'Enabled automatic OOB',
 'category': 'INFO'}